# Statistical Analysis and Classification/Prediction Models
----

This notebook covers examples of some common statistical and machine learning models and techniques.  It does not teach the theory behind the models and techniques.  

We'll look at three modules:
* [`scipy.stats`](https://docs.scipy.org/doc/scipy-0.19.1/reference/tutorial/stats.html)
* [`statsmodels`](http://www.statsmodels.org)
* [`scikit-learn`](http://scikit-learn.org/stable/)

All of the packages come with the Anaconda distribution of Python, so most of you should already have them installed. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Probability Distributions and Basic Tests

`scipy.stats` has objects for major probability distributions: https://docs.scipy.org/doc/scipy-0.19.1/reference/stats.html#module-scipy.stats

The distributions have similar functions and methods depending on the type of distribution (e.g. continuous, discrete)

In [ ]:
from scipy.stats import expon

In [ ]:
expon.stats(moments='mvsk') # mean, var, skew, kurt

Plot the pdf.  Get 100 evenly spaced values covering the range of 0 of the distribution probability to 99% (since 100% is at infinity)

In [ ]:
x = np.linspace(expon.ppf(0), expon.ppf(.99), 100) #ppf is percent point function
fig, ax = plt.subplots()
ax.plot(x, expon.pdf(x), 'r-', lw=5, alpha=0.6, label='expon pdf');

We can change the mean and standard deviation with the `loc` and `scale` parameters

In [ ]:
myloc = 2
myscale = 1
x = np.linspace(expon.ppf(0, myloc, myscale), expon.ppf(.99, myloc, myscale), 100) #ppf is percent point function
fig, ax = plt.subplots()
ax.plot(x, expon.pdf(x, myloc, myscale), 'r-', lw=5, alpha=0.6, label='expon pdf');

In [ ]:
print(expon.mean(), expon.var())
print(expon.mean(2,2), expon.var(2, 2))

You can get random samples from the distributions:

In [ ]:
expon.rvs(size=10)

### Statistical Tests

There are methods for conducting t-tests.  See: [Student's t-distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution)

In [ ]:
from scipy.stats import ttest_ind, norm
a = norm.rvs(size=100)
b = norm.rvs(.1, size=100)
ttest_ind(a, b)

Also methods for correlation that return the correlation coefficient and the two-tailed p-value on the test for no correlation; See [Pearson Correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

In [ ]:
a = norm.rvs(size=100)
b = a + 2 * norm.rvs(size=100)
plt.scatter(a,b)
stats.pearsonr(a, b)

### Transforming Data

`scipy.stats` also includes methods for transforming data, like computing the [z-score](https://en.wikipedia.org/wiki/Standard_score) or converting data to ranks.

In [ ]:
a = norm.rvs(2,5,size=20)
b = stats.zscore(a)
print(b.mean(), b.std())

In [ ]:
pd.DataFrame({'orig':a, 'zscore':b}).plot('orig','zscore',kind='scatter',
                                         xlim=(-10,10), ylim=(-10,10))

In [ ]:
ranked = pd.DataFrame({"a":a, "rank":stats.rankdata(a)})
ranked.head(10)

In [ ]:
ranked.plot("rank", "a", kind="scatter")

### Missing Data

There are also functions for dealing with arrays with missing data (NumPy masked arrays) in `scipy.stats.mstats`.

## Regression 

Let's start with OLS (ordinary least squares) regression.  OLS regression finds the line through the data the minimizes the sum of the squared errors.  It is used in many fields to identify factors (independent variables) that help explain variance in a continuous dependent variable.  In a machine learning context, it is used to predict continuous outcome variables.  

### `statsmodels`

`statmodels` includes many models for variations on linear regression.  It has an interface similar to R in many respects.

#### Get Data

We'll use a data set that's available in R.  `statsmodels` can read in data sets from R packages.

In [ ]:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "car") #first is name of data, second is package name
print(duncan_prestige.__doc__)

In [ ]:
duncan_prestige.data.head() # data is in the .data attribute

Let's look at the relationship between prestige and income to start.

In [ ]:
duncan_prestige.data.plot("prestige","income", kind="scatter")

#### OLS Linear Regression

Run an ordinary least squares regression for these two variables.  Exercises for NumPy also had you do this, but here we'll use `statsmodels`, which gives us a nice summary of the results.

Remember that we imported the models module as `smf` above.

`statsmodels` allows the forumla syntax that you'll also see in R:

```
y ~ x1 + x2
```

Or you could use NumPy arrays instead (2D array for independent variables and 1D for dependent variable)

In [ ]:
results = smf.ols('income ~ prestige', data=duncan_prestige.data).fit()
print(results.summary()) ## without "print" you get tables

This is a fairly strong relationship (for social science anyway).  But in the plot above, it looks like there might be something else going on in the data.  

We can use Seaborn to see if type of occupation might matter:

In [ ]:
sns.lmplot(x="prestige", y="income", hue="type", data=duncan_prestige.data)

Looks like there may be different slopes and intercepts.  Let's revise the model to allow for that.  

To create an interaction term, use `*`.  It will automatically add in both variables by themselves, and the two variables multiplied by each other.  `statsmodels` will automatically convert categorical variables to individual indicators first (it will choose one category as the base).

In [ ]:
results = smf.ols('income ~ prestige*type', data=duncan_prestige.data).fit()
print(results.summary())

What about education?

In [ ]:
results = smf.ols('income ~ prestige*type + education', data=duncan_prestige.data).fit()
print(results.summary())

### scikit-learn

Let's do the same regression model as above, but with scikit-learn instead.  We'll have to construct the matrix of explanatory X variables ourselves first.

In [ ]:
X = duncan_prestige.data[["prestige","education","type"]]
X = pd.get_dummies(X) # create dummy variables for type of profession
print(X.head())

Create interaction terms

In [ ]:
X['prestige-prof'] = X.prestige*X.type_prof
X['prestige-wc'] = X.prestige*X.type_wc
X = X[["type_prof","type_wc","prestige","prestige-prof","prestige-wc","education"]]

Run the model.  We can get the coefficients, but there isn't a method for a nice summary of results.

Why no summary?  Scikit-learn is focused on machine learning models, where you mostly care about the predictions, not the statistics of the coefficients.

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X, duncan_prestige.data.income)
print(reg.coef_)
print(reg.intercept_)

Compare these to the results from `statsmodels` above.  They are the same.   Use `np.isclose()` because they may not be the same to full precision.  Remove the intercept from `results.params` before comparing.

In [ ]:
np.isclose(reg.coef_,results.params[1:].tolist()) 

We can get some information about the fit of our model, for example $R^2$, which tells us how much variation in the dependent variable we're explaining: [`sklearn.metrics.r2_score()`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score)

In [ ]:
sklearn.metrics.r2_score(duncan_prestige.data.income, reg.predict(X))

## Classification

If the outcome variable we want to predict or explain is not continuous, then classification models may be useful.  There are variations on OLS regression models for binary (logistic regression) and categorical (multinomial regression) dependent variables, but there are also many other classification models.

Regression-type models are available in `statsmodels`.

Scikit-learn has many classification models available for different situations and types of data.  One of the benefits of Scikit-learn is that all models, even regression like we used above, follow the same basic pattern to run:

* Import a model 
* Create an instance of the model object specify any necessary parameters
* Fit the model with the `.fit()` function
* Get predicted values with the `.predict()` function

Before this, you'll need to get your data ready.  After this, you'll have to assess how well your model did predicting the data.

### Example: Predicting Breast Cancer

#### Get the Data

Data URL: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data

There is missing data denoted with `?`

Column descriptions, from [source](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names):

```
   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
  ```

In [ ]:
cancer = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",
                    names=['sample','clump','unif_size','unif_shape','adhesion',
                          'single_size','nuclei','chromatin','normal','mitoses','class'],
                    na_values=['?'])
cancer.head()

In [ ]:
print(cancer.shape)

In [ ]:
cancer = cancer.set_index('sample')
cancer['malignant'] = cancer['class'].map({2:0,4:1})
cancer.head()

Find missing values, because Scikit-learn doesn't support them

In [ ]:
cancer[cancer.isnull().any(axis=1)]

Let's look at the distribution of each variable by malignancy.

In [ ]:
from IPython.display import display # so we can output multiple plots
for var in cancer.columns.values[:-2]:
    display(sns.factorplot(y=var,data=cancer, col="malignant", kind="box"))

### Try to Predict

#### Logistic regression

In [ ]:
logit = linear_model.LogisticRegression()
X = cancer.iloc[:,:-2] # omit original class var and recoded malignant var
del X['nuclei'] # because it has missing data
logit.fit(X, cancer['malignant'])

print("Predicted wrong: {}".format((cancer['malignant']-logit.predict(X)).abs().sum()))
print("Total: {}".format(cancer.shape[0]))
print("Overall accuracy: {}".format(logit.score(X, cancer['malignant'])))

This isn't too bad.  But we probably care more about misclassifying malignant tumors as benign than the other way around.  Let's look at predictions by category.

In [ ]:
pd.crosstab(cancer['malignant'], logit.predict(X))

Bottom left cell are the cases where the tumor was malignant but the model did not predict malignancy.  This is also called a confusion matrix, and there's a function for it:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(cancer['malignant'], logit.predict(X))

What we really would care about is not how well the model predicts on the data we have, but how well it predicts on new data.  To test this, we'd train a model on a subset of the data and then evaluate it on a different set of the data.  

Scikit-learn has functions to help us with this.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, cancer['malignant'], test_size=0.3, random_state=42)
logit.fit(X_train, y_train)
logit.score(X_test, y_test)

So the model using only 70% of the data was able to predict with about the same accuracy as the full model on a 30% test sample.  Does this generalize to other splits of the data?  We can use [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) to hold out part of the data, train on the rest, and then test against the held out part.  Do this for multiple splits of the data.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logit, X, cancer['malignant'], cv=10) # cv is number of times to do this/splits
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

The +/- is a 95% confidence interval.

#### Naive Bayes

Now try a [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html) model ([another explanation](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html)).  They tend to be good out-of-the-box classifiers.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X, cancer['malignant'])

scores = cross_val_score(gnb, X, cancer['malignant'], cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Similar, slightly worse.

#### Decision Tree and Random Forest

[Decision trees](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb) are a non-parametric model for classification.  One of the biggest issues with decision trees is [over-fitting](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html).  To deal with that, sometimes you need to prune a tree.  A general approach is instead of using a single decision tree to predict, you create a random forest of trees to vote on the prediction.

Same steps with scikit-learn as before:

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, cancer['malignant'])

scores = cross_val_score(clf, X, cancer['malignant'], cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

But the interesting thing about trees is that we can look at them.  Scikit-learn doesn't make this particularly easy.  We'll need an extra Python module (pydotplus) for drawing graphs (the network/tree kind) and an underlying system library ([Graphviz](http://www.graphviz.org/)) for generating the graph.  Until you have this, the code below won't run, so it's saved as a raw cell (not executable).

 ![tree](https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/models/tree.png)

Close-up of leaves.  Algorithm makes decisions about which variable and which cut-point to use based on [gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) (there are other options).

![tree closeup](https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/models/treecloseup.png)

Refit, setting a max depth to try to curb overfitting:

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=4)
clf = clf.fit(X, cancer['malignant'])

scores = cross_val_score(clf, X, cancer['malignant'], cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

But it would be better to use a bunch of trees: [RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=0)
scores = cross_val_score(clf, X, cancer['malignant'], cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))